In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install datasets 

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 136 kB 49.2 MB/s 
     |████████████████████████████████| 212 kB 50.5 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 127 kB 51.9 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
     |████████████████████████████████| 144 kB 46.8 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from datasets import list_datasets, load_dataset
from pprint import pprint

datasets_list = list_datasets() 
dataset = load_dataset('imdb','plain_text')  

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import numpy as np

#generate data and labels, combine test and train sets, and shuffle them randomly
subset_data = [i for i in dataset['train'][:]['text']]
subset_data.extend([i for i in dataset['test'][:]['text']])
data = np.array(subset_data)
sentiment = [i for i in dataset['train'][:]['label']]
sentiment.extend([i for i in dataset['test'][:]['label']])
sentiment = np.array(sentiment)
permutation = np.random.permutation(len(data))
data_shuffled = data[permutation]
label_shuffled = sentiment[permutation]

In [ ]:
#tokenize and tag the data
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_shuffled)]


In [ ]:
#intialize model and vocab
max_epochs = 10
vec_size = 500
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
#train doc2vec model

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data[:10000],
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [ ]:
#save model for future use
model.save("d2v.model")
print("Model Saved")

Model Saved


In [ ]:
from gensim import utils
model = utils.SaveLoad.load('d2v.model')
#test model inference
print(model.infer_vector(['violent', 'means', 'to', 'destroy', 'the','organization']))

[-0.00639921  0.03812475 -0.05181699 -0.01639308  0.03394845 -0.05974185
 -0.06099918 -0.01021937 -0.03239402 -0.04859616 -0.02313961 -0.07680973
  0.06352254 -0.05254771 -0.07747042  0.1142513  -0.07109381  0.12424589
  0.07333766 -0.01626833 -0.03564634 -0.09328228  0.10227896 -0.08944736
 -0.00494694 -0.010035   -0.0701941   0.10607354 -0.09104972  0.05183989
 -0.05602399 -0.01342105 -0.08205913  0.00560406 -0.04512663  0.02024725
  0.05983262 -0.03860375  0.00051075  0.15835841 -0.06055825  0.02300882
 -0.08638741 -0.06521934 -0.09626041  0.14101355  0.0654083  -0.05939839
 -0.09085424  0.01030518 -0.1184643  -0.13154058  0.10099517 -0.09281763
 -0.00889296  0.08552821 -0.04540504 -0.04042365 -0.04874481 -0.05233584
  0.04316134 -0.05023036 -0.01435248  0.07226392 -0.05983528  0.03642747
  0.00292915  0.09682667  0.09266184 -0.0512856  -0.05880718  0.02779866
  0.03086051  0.01995465 -0.0813643   0.05839914 -0.07003931 -0.10229954
 -0.06209341 -0.05403804 -0.13363574 -0.0964598   0

In [ ]:
#create embeddings
X = []

for doc in data_shuffled:
  X.append(model.infer_vector(doc.split()))
X = np.array(X)

np.savetxt('embeddings.txt', X)

In [ ]:
#define an analysis function for model metrics
def analyze(y_true, y_pred):
  c_matrix = confusion_matrix(y_true, y_pred)
  print("Confusion Matrix:")
  print(c_matrix)
  (tn, fp, fn, tp) = c_matrix.ravel()
  tn_rate = tn/(tn+fp)
  tp_rate = tp/(tp+fn)
  precision = tp/(tp+fp)
  f1 = 2*((precision*tp_rate)/(precision+tp_rate))
  print("True Positive Rate: {:.2f}".format(tp_rate))
  print("True Negative Rate: {:.2f}".format(tn_rate))
  print("Precision: {:.2f}".format(precision))
  print("F1 Score: {:.2f}".format(f1))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
from sklearn.metrics import confusion_matrix, f1_score

#explore different Kmeans & GMM clustering train-test splits
thresholds = [10000, 15000, 20000, 25000, 30000, 35000, 40000]

for test_train_threshold in thresholds:

  #supervised
  #gnb = GaussianNB()
  #gnb.fit(X[:test_train_threshold], label_shuffled[:test_train_threshold])

  #log = LogisticRegression()
  #log.fit(X[:test_train_threshold], label_shuffled[:test_train_threshold])

  #unsupervised
  kmeans = KMeans(n_clusters=2, random_state=0).fit(X[:test_train_threshold])
  gmm = GaussianMixture(n_components=2,covariance_type='full').fit(X[:test_train_threshold])

  predictions_kmeans = kmeans.predict(X[test_train_threshold:])
  predictions_gmm = gmm.predict(X[test_train_threshold:])
  predictions_kmeans2 = [not x for x in predictions_kmeans]
  predictions_gmm2 = [not x for x in predictions_kmeans]
  #predictions_nb = gnb.predict(X[test_train_threshold:])
  #predictions_log = log.predict(X[test_train_threshold:])
  print("threshold: " + str(test_train_threshold))
  print("kmeans: ")
  analyze(label_shuffled[test_train_threshold:], predictions_kmeans)
  analyze(label_shuffled[test_train_threshold:], predictions_kmeans2)
  print("gmm: ")
  analyze(label_shuffled[test_train_threshold:], predictions_gmm)
  analyze(label_shuffled[test_train_threshold:], predictions_gmm2)
  #print("naive bayes: ")
  #analyze(label_shuffled[test_train_threshold:], predictions_nb)
  #print("logistic regression: ")
  #analyze(label_shuffled[test_train_threshold:], predictions_log)

threshold: 10000
kmeans: 
Confusion Matrix:
[[16597  3448]
 [16093  3862]]
True Positive Rate: 0.19
True Negative Rate: 0.83
Precision: 0.53
F1 Score: 0.28
Confusion Matrix:
[[ 3448 16597]
 [ 3862 16093]]
True Positive Rate: 0.81
True Negative Rate: 0.17
Precision: 0.49
F1 Score: 0.61
gmm: 
Confusion Matrix:
[[11505  8540]
 [11351  8604]]
True Positive Rate: 0.43
True Negative Rate: 0.57
Precision: 0.50
F1 Score: 0.46
Confusion Matrix:
[[ 3448 16597]
 [ 3862 16093]]
True Positive Rate: 0.81
True Negative Rate: 0.17
Precision: 0.49
F1 Score: 0.61
threshold: 15000
kmeans: 
Confusion Matrix:
[[ 3514 14053]
 [ 3812 13621]]
True Positive Rate: 0.78
True Negative Rate: 0.20
Precision: 0.49
F1 Score: 0.60
Confusion Matrix:
[[14053  3514]
 [13621  3812]]
True Positive Rate: 0.22
True Negative Rate: 0.80
Precision: 0.52
F1 Score: 0.31
gmm: 
Confusion Matrix:
[[8444 9123]
 [8456 8977]]
True Positive Rate: 0.51
True Negative Rate: 0.48
Precision: 0.50
F1 Score: 0.51
Confusion Matrix:
[[14053  351